In [ ]:
%load_ext Cython
%%cython
import numpy as np
#import astropy.units as u
#import astropy.coordinates as coord
#from astropy.coordinates import SkyCoord
cimport numpy as np
cimport cython
from libc.math cimport sqrt,sin,cos,acos,asin,atan2
from libc.math cimport M_PI as pi
@cython.boundscheck(False)
@cython.wraparound(False)
def error_toGalcen(double[:] ra, double[:] dec, double[:] parallax, double[:] pmra, double[:] pmdec, double[:] ddot,double[:] sigma):
        """
        Propagates network's sigma to Galactocentric cartesian frame 
        """
        cdef double alpha_NGP = 192.85948*pi/180
        cdef double delta_NGP = 27.12825*pi/180
        cdef double theta = 122.932*pi/180
        cdef double k = 4.74047

        cdef np.ndarray[double, ndim=2, mode='c'] T1 = np.array([[cos(theta),sin(theta),0],[sin(theta),-cos(theta),0],[0,0,1]])
        cdef np.ndarray[double, ndim=2, mode='c'] T2 = np.array([[-sin(delta_NGP),0,cos(delta_NGP)],[0,1,0],[cos(delta_NGP),0,sin(delta_NGP)]])
        cdef np.ndarray[double, ndim=2, mode='c'] T3 = np.array([[cos(alpha_NGP),sin(alpha_NGP),0],[-sin(alpha_NGP),cos(alpha_NGP),0],[0,0,1]])

        cdef np.ndarray[double, ndim=2, mode='c'] T = np.matmul(np.matmul(T1,T2),T3)

        cdef np.ndarray[double, ndim=2, mode='c'] A1 = np.empty((3,3),dtype=np.float)
        cdef np.ndarray[double, ndim=2, mode='c'] A2 = np.empty((3,3),dtype=np.float)
        cdef np.ndarray[double, ndim=2, mode='c'] A = np.empty((3,3),dtype=np.float)

        cdef int nstars = len(sigma)
        cdef np.ndarray[double, ndim=2, mode='c'] COV_VRAB = np.empty((3,3),dtype=np.float)
        cdef np.ndarray[double, ndim=2, mode='c'] COV_UVW = np.empty((3,3),dtype=np.float)
        cdef np.ndarray[double, ndim=1, mode='c'] COV_U = np.empty(nstars,dtype=np.float)
        cdef np.ndarray[double, ndim=1, mode='c'] COV_V = np.empty(nstars,dtype=np.float)
        cdef np.ndarray[double, ndim=1, mode='c'] COV_W = np.empty(nstars,dtype=np.float)

        for i in range(nstars):
                A1 = np.array([[cos(ra[i]),-sin(ra[i]),0],[sin(ra[i]),cos(ra[i]),0],[0,0,1]])
                A2 = np.array([[cos(dec[i]),0,-sin(dec[i])],[0,1,0],[sin(dec[i]),0,cos(dec[i])]])
                A = np.matmul(A1,A2)
                #print(A)
                COV_VRAB = np.zeros_like(COV_VRAB)
                COV_VRAB[0,0] = sigma[i]
                
                #print(np.matmul(np.matmul(T,A),np.matmul(COV_VRAB,np.matmul(np.transpose(A),np.transpose(T)))).shape)
                COV_UVW = np.matmul(np.matmul(T,A),np.matmul(COV_VRAB,np.matmul(np.transpose(A),np.transpose(T))))
                
                COV_U[i] = COV_UVW[0,0]
                COV_V[i] = COV_UVW[1,1]
                COV_W[i] = COV_UVW[2,2]
        print(COV_UVW)
        return COV_U, COV_V, COV_W
def error_toGalcen_sph(double[:] ra, double[:] dec, double[:] b, double[:] l, double[:] parallax, double[:] pmra, double[:] pmdec, double[:] theta, double[:] phi, double[:] ddot,double[:] sigma):
        """
        Propagates network error to galactocentric spherical coordinates
        """
        cdef double alpha_NGP = 192.85948*pi/180
        cdef double delta_NGP = 27.12825*pi/180
        #cdef double theta = 122.932*pi/180
        cdef double k = 4.74047
        cdef np.ndarray[double, ndim=1 , mode='c'] solar_corr = np.array([11.1, 239.08, 7.25])
        cdef double galcen_distance = 8.0004 # in kpc
        cdef double z_sun = 0.015 # in kpc
        
        cdef np.ndarray[double, ndim=2, mode='c'] P = np.empty((3,3),dtype=np.float)
        cdef np.ndarray[double, ndim=2, mode='c'] mat_1 = np.empty((3,3),dtype=np.float)
        cdef np.ndarray[double, ndim=2, mode='c'] mat_2 = np.empty((3,3),dtype=np.float)
        cdef np.ndarray[double, ndim=2, mode='c'] mat_3 = np.empty((3,3),dtype=np.float)
        cdef np.ndarray[double, ndim=2, mode='c'] mat_4 = np.empty((3,3),dtype=np.float)
        cdef np.ndarray[double, ndim=2, mode='c'] M = np.empty((3,3),dtype=np.float)
        
        cdef int nstars = len(sigma)
        cdef np.ndarray[double, ndim=2, mode='c'] COV_VLOSAD = np.empty((3,3),dtype=np.float)
        cdef np.ndarray[double, ndim=2, mode='c'] COV_VRTP = np.empty((3,3),dtype=np.float)
        cdef np.ndarray[double, ndim=1, mode='c'] COV_VR = np.empty(nstars,dtype=np.float)
        cdef np.ndarray[double, ndim=1, mode='c'] COV_VTHETA = np.empty(nstars,dtype=np.float)
        cdef np.ndarray[double, ndim=1, mode='c'] COV_VPHI = np.empty(nstars,dtype=np.float)
        
        cdef double sin_theta_sol = z_sun/galcen_distance
        cdef double cos_theta_sol = np.sqrt(1. - sin_theta_sol**2)
        cdef np.ndarray[double, ndim=2, mode='c'] mat_sol = np.array([[ cos_theta_sol, 0, sin_theta_sol],[0,1,0],[-sin_theta_sol, 0, cos_theta_sol]])

        for i in range(nstars):
            COV_VLOSAD = np.zeros_like(COV_VLOSAD)
            COV_VLOSAD[0,0] = sigma[i]
            cos_phi_conv = (sin(delta_NGP) - sin(dec[i]) * sin(b[i])) / (cos(dec[i]) * cos(b[i]))
            sin_phi_conv = sin(ra[i] - alpha_NGP) * cos(delta_NGP) / cos(b[i])
            P = np.array([[1,0,0],[0,cos_phi_conv, sin_phi_conv],[0,-sin_phi_conv, cos_phi_conv]])
            mat_1 = np.array([[cos(theta[i]), 0, sin(theta[i])],[0,1,0],[-sin(theta[i]), 0, cos(theta[i])]])
            mat_2 = np.array([[cos(phi[i]), sin(phi[i]), 0],[-sin(phi[i]), cos(phi[i]), 0],[0,0,1]])
            mat_3 = np.array([[cos(l[i]), -sin(l[i]), 0], [sin(l[i]),  cos(l[i]), 0],[0,0,1]])
            mat_4 = np.array([[cos(b[i]), 0, -sin(b[i])], [0,1,0],[sin(b[i]), 0, cos(b[i])]])
        
            M = np.matmul(np.matmul(np.matmul(np.matmul(mat_1,mat_2),mat_sol),mat_3),mat_4)
            COV_VRTP = np.matmul(np.matmul(M,P),np.matmul(COV_VLOSAD,np.matmul(np.transpose(P),np.transpose(M))))
            COV_VR[i] = COV_VRTP[0,0]
            COV_VTHETA[i] = COV_VRTP[1,1]
            COV_VPHI[i] = COV_VRTP[2,2]
        print(COV_VRTP)
        return COV_VR, COV_VTHETA, COV_VPHI

In [ ]:
error_toGalcen_sph(np.ones(10), np.ones(10), np.ones(10), np.ones(10), np.ones(10), np.ones(10), np.ones(10),np.ones(10),np.ones(10),np.ones(10),np.ones(10))